In [7]:
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from bs4 import BeautifulSoup

In [8]:
loader = PDFMinerPDFasHTMLLoader("Input/adversarial_diffusion_distillation.pdf")
data = loader.load()[0]

print(data)

page_content='<html><head>\n<meta http-equiv="Content-Type" content="text/html">\n</head><body>\n<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:612px; height:792px;"></span>\n<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>\n<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:195px; top:106px; width:203px; height:14px;"><span style="font-family: NimbusRomNo9L-Medi; font-size:14px">Adversarial Diffusion Distillation\n<br></span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:99px; top:146px; width:53px; height:11px;"><span style="font-family: NimbusRomNo9L-Regu; font-size:11px">Axel Sauer\n<br></span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:183px; top:146px; width:79px; height:11px;"><span style="font-family: NimbusRomNo9L-Regu; font-size:11px">Dominik Lorenz\n<br></span></div><div style="position:absolute; borde

In [26]:
soup = BeautifulSoup(data.page_content,'html.parser')
content = soup.find_all('div')

print(content)

[<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>, <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:195px; top:106px; width:203px; height:14px;"><span style="font-family: NimbusRomNo9L-Medi; font-size:14px">Adversarial Diffusion Distillation
<br/></span></div>, <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:99px; top:146px; width:53px; height:11px;"><span style="font-family: NimbusRomNo9L-Regu; font-size:11px">Axel Sauer
<br/></span></div>, <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:183px; top:146px; width:79px; height:11px;"><span style="font-family: NimbusRomNo9L-Regu; font-size:11px">Dominik Lorenz
<br/></span></div>, <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:294px; top:146px; width:92px; height:11px;"><span style="font-family: NimbusRomNo9L-Regu; font-size:11px">Andreas Blattmann
<br/></span></div>, <div style="

In [16]:
import re
cur_fs = None
cur_text = ''
snippets = []   # first collect all snippets that have the same font size
for c in content:
    sp = c.find('span')
    if not sp:
        continue
    st = sp.get('style')
    if not st:
        continue
    fs = re.findall('font-size:(\d+)px',st)
    if not fs:
        continue
    fs = int(fs[0])
    if not cur_fs:
        cur_fs = fs
    if fs == cur_fs:
        cur_text += c.text
    else:
        snippets.append((cur_text,cur_fs))
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text,cur_fs))

print(snippets)
# Note: The above logic is very straightforward. One can also add more strategies such as removing duplicate snippets (as
# headers/footers in a PDF appear on multiple pages so if we find duplicates it's safe to assume that it is redundant info)

Page 1
-----------------------
Adversarial Diffusion Distillation

-----------------------
Axel Sauer

-----------------------
Dominik Lorenz

-----------------------
Andreas Blattmann

-----------------------
Robin Rombach

-----------------------
Code: https://github.com/Stability-AI/generative-models Model weights: https://huggingface.co/stabilityai/

-----------------------
Stability AI

-----------------------
Figure 1. Generating high-ﬁdelity 5122 images in a single step. All samples are generated with a single U-Net evaluation trained with
adversarial diffusion distillation (ADD).

-----------------------
Abstract

-----------------------
1. Introduction

-----------------------
We introduce Adversarial Diffusion Distillation (ADD), a
novel training approach that efﬁciently samples large-scale
foundational image diffusion models in just 1–4 steps while
maintaining high image quality. We use score distillation
to leverage large-scale off-the-shelf image diffusion models
as a teac

In [27]:
from langchain.docstore.document import Document
cur_idx = -1
semantic_snippets = []
# Assumption: headings have higher font size than their respective content
for s in snippets:
    if s[0].lower == 'references': continue
    # if current snippet's font size > previous section's heading => it is a new heading
    if not semantic_snippets or s[1] > semantic_snippets[cur_idx].metadata['heading_font']:
        metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
        metadata.update(data.metadata)
        semantic_snippets.append(Document(page_content='',metadata=metadata))
        cur_idx += 1
        continue

    # if current snippet's font size <= previous section's content => content belongs to the same section (one can also create
    # a tree like structure for sub sections if needed but that may require some more thinking and may be data specific)
    if not semantic_snippets[cur_idx].metadata['content_font'] or s[1] <= semantic_snippets[cur_idx].metadata['content_font']:
        print(f"second if: {s}")
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata['content_font'] = max(s[1], semantic_snippets[cur_idx].metadata['content_font'])
        continue

    # if current snippet's font size > previous section's content but less than previous section's heading than also make a new
    # section (e.g. title of a PDF will have the highest font size but we don't want it to subsume all sections)
    metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
    metadata.update(data.metadata)
    semantic_snippets.append(Document(page_content='',metadata=metadata))
    cur_idx += 1

Prev Size: 14
second if: ('Axel Sauer\nDominik Lorenz\nAndreas Blattmann\nRobin Rombach\n', 11)
second if: ('Code: https://github.com/Stability-AI/generative-models Model weights: https://huggingface.co/stabilityai/\n', 8)
second if: ('Stability AI\n', 11)
second if: ('Figure 1. Generating high-ﬁdelity 5122 images in a single step. All samples are generated with a single U-Net evaluation trained with\nadversarial diffusion distillation (ADD).\n', 8)
second if: ('Abstract\n1. Introduction\n', 11)
second if: ('We introduce Adversarial Diffusion Distillation (ADD), a\nnovel training approach that efﬁciently samples large-scale\nfoundational image diffusion models in just 1–4 steps while\nmaintaining high image quality. We use score distillation\nto leverage large-scale off-the-shelf image diffusion models\nas a teacher signal in combination with an adversarial loss\nto ensure high image ﬁdelity even in the low-step regime\nof one or two sampling steps. Our analyses show that our\nmodel cl

In [20]:
print(len(semantic_snippets))
for i in semantic_snippets:
    print(len(i.page_content))

6
49366
0
32
437
0
1866
